## Baseline score -  randomly guessing given that we know that 20% of the data is popular

In [1]:
import random
import pandas as pd
import numpy as np
import pickle
import matplotlib.pyplot as plt
from datetime import datetime as dt
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix, make_scorer, recall_score, precision_score, f1_score

pd.set_option("display.max_colwidth",999)
pd.set_option("display.max_rows",999)
pd.set_option("display.max_columns",999)
np.set_printoptions(suppress=True)

In [244]:
def randomly_guessing(n_simulations = 10):
    
    count = 0 #
    #keep track of 
    accuracy_baseline = np.zeros(shape=(1,9*n_simulations)) #we have 9 test sets for 2017. Multiply by 9 for each simulation.  
    recall_baseline = np.zeros(shape=(1,9*n_simulations))
    precision_baseline = np.zeros(shape=(1,9*n_simulations))
    f1_baseline = np.zeros(shape=(1,9*n_simulations))
    
    for i in range(n_simulations):
        my_list = [True for x in range(2)] +  [False for x in range(8)]
        start_month = 1
        end_month = 4
        while end_month <13:
            y_test = df_sf_2017[df_sf_2017['month'] == end_month]['popular']
            y_pred = pd.Series(random.choice(my_list) for x in range(y_test.size))
            accuracy_baseline[0][count] = accuracy_score(y_test, y_pred)
            recall_baseline[0][count] = recall_score(y_test, y_pred)
            precision_baseline[0][count] = precision_score(y_test, y_pred)
            f1_baseline[0][count] = f1_score(y_test, y_pred)
            count+=1
            start_month += 1
            end_month += 1
            
    return accuracy_baseline, recall_baseline, precision_baseline, f1_baseline


In [235]:
accuracy_baseline, recall_baseline, precision_baseline, f1_baseline = randomly_guessing(n_simulations = 1000)
print(accuracy_baseline.mean())
print(recall_baseline.mean()) 
print(precision_baseline.mean()) 
print(f1_baseline.mean())

0.6738568003192421
0.19992382244534446
0.2098987145097955
0.20396154681655437


In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/ubuntu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [62]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/ubuntu/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [86]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/ubuntu/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

# Load data

In [9]:
# df_sf_2017 = pickle.load(open('../data_sf_2017.p', 'rb'))

In [2]:
df_sf_2017 = pd.read_json('df_sf_2017')

In [11]:
len(df_sf_2017)

117262

# NLP

In [3]:
import nltk
from nltk.tokenize import RegexpTokenizer, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import time
from sklearn.metrics import accuracy_score, confusion_matrix, recall_score, precision_score, f1_score

In [4]:
df_sf_2017.space.fillna(value='None', inplace=True)

In [5]:
df_sf_2017["space_new"] = df_sf_2017['space'].str.replace('[^\w\s]','')

In [215]:
# tf_vectorizer_train.vocabulary_

{'floor': 30,
 'breath': 15,
 'taking': 86,
 'view': 96,
 'easy': 25,
 'transportation': 91,
 'downtown': 24,
 'great': 35,
 'neighborhood': 55,
 'san': 76,
 'francisco': 31,
 'safe': 75,
 'free': 32,
 'parking': 59,
 'located': 46,
 'geographically': 34,
 'center': 17,
 'city': 18,
 'public': 68,
 'available': 10,
 '15': 1,
 'minute': 51,
 'private': 67,
 'bathroom': 11,
 'room': 74,
 'street': 83,
 'lovely': 48,
 'noe': 57,
 'valley': 94,
 'access': 5,
 'muni': 54,
 'market': 49,
 'perfectly': 63,
 'enjoy': 26,
 'peaceful': 61,
 'environment': 28,
 'want': 97,
 'close': 20,
 'shopping': 81,
 'restaurant': 72,
 'nightlife': 56,
 'block': 14,
 'j': 41,
 'train': 90,
 '20m': 3,
 'ride': 73,
 '3': 4,
 'mission': 52,
 'apartment': 8,
 '2': 2,
 'bedroom': 12,
 '1': 0,
 'recently': 70,
 'remodeled': 71,
 'modern': 53,
 'kitchen': 42,
 'dishwasher': 23,
 'washerdryer': 98,
 'building': 16,
 'use': 93,
 'best': 13,
 'hill': 38,
 'climb': 19,
 'sunny': 85,
 'airy': 6,
 'minimalist': 50,
 'terr

## LemmaTokenizer

In [6]:
from nltk import word_tokenize          
from nltk.stem import WordNetLemmatizer 
class LemmaTokenizer(object):
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, articles):
        return [self.wnl.lemmatize(t) for t in word_tokenize(articles)]

## split data - running NLP on "access" column

In [25]:
def split_data(start_month, end_month):
    df_X_train = df_sf_2017[(df_sf_2017['month'] >= start_month) & (df_sf_2017['month'] < end_month)]['space_new']
    y_train = df_sf_2017[(df_sf_2017['month'] >= start_month) & (df_sf_2017['month'] < end_month)]['popular']

    df_X_test = df_sf_2017[df_sf_2017['month'] == end_month]['space_new']
    y_test = df_sf_2017[df_sf_2017['month'] == end_month]['popular']
    
    return df_X_train, y_train, df_X_test, y_test

## Run CountVectorizer

In [14]:
%%time
def run_tf_vec(df_X_train, df_X_test):
    tf_vectorizer_train = CountVectorizer(tokenizer=LemmaTokenizer(), stop_words='english').fit(df_X_train)
    X_train = tf_vectorizer_train.transform(df_X_train)
    tf_vectorizer_test = CountVectorizer(tokenizer=LemmaTokenizer(), stop_words='english', vocabulary = tf_vectorizer_train.vocabulary_).fit(df_X_test)
    X_test = tf_vectorizer_test.transform(df_X_test)
    return X_train, X_test, tf_vectorizer_train

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 6.44 µs


In [95]:
X_train

<26526x12784 sparse matrix of type '<class 'numpy.int64'>'
	with 1848230 stored elements in Compressed Sparse Row format>

In [8]:
def predict_tf_nb(X_train, y_train, X_test, y_test):
    from sklearn.naive_bayes import MultinomialNB
    nb = MultinomialNB()
    nb.fit(X_train, y_train)
    pickle.dump(nb, open('nb_model'+ str(model_num) + '.p', 'wb'))
    preds = nb.predict(X_test)
    scores_tf_nb[0][model_num] = accuracy_score(y_test, preds)
    scores_tf_nb[1][model_num] = recall_score(y_test, nb.predict(X_test))
    scores_tf_nb[2][model_num] = precision_score(y_test, nb.predict(X_test))
    scores_tf_nb[3][model_num] = f1_score(y_test, nb.predict(X_test))
    return scores_tf_nb

In [9]:
# def grid_searching (param_grid, model):

#     grid_search = GridSearchCV(model, 
#                                param_grid=param_grid, cv=5, 
#                                n_jobs=-1, scoring=make_scorer(f1_score))
#     fit = grid_search.fit(X_train, y_train)
#     predicted = fit.predict(X_test)
#     return grid_search.best_params_


## With the CountVectorizer, run with RandomForest 

In [16]:
def predict_tf_rf(X_train, y_train, X_test, y_test):
    
    rf = RandomForestClassifier(n_estimators = 500, 
                                n_jobs = -1, 
                                random_state = 0, 
                                max_depth = 10)
    rf.fit(X_train, y_train)
    predicted = rf.predict(X_test)
#     pickle.dump(rf, open('rf_nlp_countvec_50' + str(model_num) + '.p', 'wb'))
    scores_tf_rf[0][model_num] = accuracy_score(y_test, predicted)
    scores_tf_rf[1][model_num] = recall_score(y_test, predicted)
    scores_tf_rf[2][model_num] = precision_score(y_test, predicted)
    scores_tf_rf[3][model_num] = f1_score(y_test, predicted)
    return scores_tf_rf

In [30]:
scores_tf_nb = np.zeros(shape=(4,9))
scores_tf_rf = np.zeros(shape=(4,9))

model_num = 0
start_month = 1
end_month = 4

In [31]:
%%time
while end_month <13:
    df_X_train, y_train, df_X_test, y_test = split_data(start_month, end_month)
    X_train, X_test, tf_vectorizer_train = run_tf_vec(df_X_train, df_X_test)
    scores_tf_nb = predict_tf_nb(X_train, y_train, X_test, y_test)
    scores_tf_rf = predict_tf_rf(X_train, y_train, X_test, y_test)
    print(start_month, end_month, model_num)
    model_num += 1
    start_month += 1
    end_month += 1
    print('tf_nb')
    print(scores_tf_nb)
    print('tf_rf')
    print(scores_tf_rf)

1 4 0
tf_nb
[[0.86741858 0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.57871537 0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.65549215 0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.61471572 0.         0.         0.         0.         0.
  0.         0.         0.        ]]
tf_rf
[[0.8200023  0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.01700252 0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.9        0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.03337454 0.         0.         0.         0.         0.
  0.         0.         0.        ]]
2 5 1
tf_nb
[[0.86741858 0.86995413 0.         0.         0.         0.
  0.         0.         0.        ]
 [0.57871537 0.57971014 0.         0.         0.         0.
  0.         0.         0.        ]
 [0.6554

## COUNT VECTORIZER NAIVE BAYES

In [22]:
accuracy_2017_tf_nb = np.mean(scores_tf_nb[0])
recall_2017_tf_nb = np.mean(scores_tf_nb[1])
precision_2017_tf_nb = np.mean(scores_tf_nb[2])
f1_score_2017_tf_nb = np.mean(scores_tf_nb[3])
print(accuracy_2017_tf_nb)
print(recall_2017_tf_nb)
print(precision_2017_tf_nb)
print(f1_score_2017_tf_nb)

NameError: name 'scores_tf_nb' is not defined

## COUNT VECTORIZER RANDOM FOREST

In [35]:
accuracy_2017_tf_rf = np.mean(scores_tf_rf[0])
recall_2017_tf_rf = np.mean(scores_tf_rf[1])
precision_2017_tf_rf = np.mean(scores_tf_rf[2])
f1_score_2017_tf_rf = np.mean(scores_tf_rf[3])
print(accuracy_2017_tf_rf)
print(recall_2017_tf_rf)
print(precision_2017_tf_rf)
print(f1_score_2017_tf_rf)

0.7938982292282903
0.020326566962152987
0.9585429344031495
0.039777603015805005


## Try running with TF-IDF

In [13]:
def run_tf_idf_vec(df_X_train, df_X_test):
    tf_idf_vectorizer_train = TfidfVectorizer(tokenizer=LemmaTokenizer(), stop_words='english')
    X_train = tf_idf_vectorizer_train.fit_transform(df_X_train).toarray()
    tf_idf_vectorizer_test = TfidfVectorizer(tokenizer=LemmaTokenizer(), stop_words='english', vocabulary = tf_idf_vectorizer_train.vocabulary_)
    X_test = tf_idf_vectorizer_test.fit_transform(df_X_test).toarray()
    return X_train, X_test, tf_idf_vectorizer_train

In [9]:
def predict_tf_idf_nb(X_train, y_train, X_test, y_test):
    from sklearn.naive_bayes import GaussianNB
    nb = GaussianNB()
    nb.fit(X_train, y_train)
    # pickle.dump(nb, open('nb_model_guassian.p', 'wb'))
    preds = nb.predict(X_test2)
    scores_tf_idf_nb[0][model_num] = accuracy_score(y_test, preds)
    scores_tf_idf_nb[1][model_num] = recall_score(y_test, preds)
    scores_tf_idf_nb[2][model_num] = precision_score(y_test, preds)
    scores_tf_idf_nb[3][model_num] = f1_score(y_test, preds)
    return scores_tf_idf_nb

## With the TF-IDF, run with RandomForest 

In [10]:
def predict_tf_idf_rf(X_train, y_train, X_test, y_test):
    rf = RandomForestClassifier(n_estimators = 10, n_jobs=-1, random_state=0, class_weight = {0:.95, 1:.05})
    rf.fit(X_train, y_train)
    predicted = rf.predict(X_test2)
#     pickle.dump(rf, open('rf_nlp_50.p', 'wb'))
    scores_tf_idf_rf[0][model_num] = accuracy_score(y_test, predicted)
    scores_tf_idf_rf[1][model_num] = recall_score(y_test, predicted)
    scores_tf_idf_rf[2][model_num] = precision_score(y_test, predicted)
    scores_tf_idf_rf[3][model_num] = f1_score(y_test, predicted)
    return scores_tf_idf_rf

In [55]:
# a = pickle.load(open('rf_nlp_countvec_500.p', 'rb'))
# a.fit(X_train, y_train)
# preds = a.predict(X_test)

In [11]:
scores_tf_idf_nb = np.zeros(shape=(4,9))
scores_tf_idf_rf = np.zeros(shape=(4,9))

model_num = 0
start_month = 1
end_month = 4

In [14]:
%%time
while end_month <13:
    df_X_train, y_train, df_X_test, y_test = split_data(start_month, end_month) 
    X_train2, X_test2, tf_idf_vectorizer_train = run_tf_idf_vec(df_X_train, df_X_test)
    scores_tf_idf_nb = predict_tf_idf_nb(X_train2, y_train, X_test2, y_test)
    scores_tf_idf_rf = predict_tf_idf_rf(X_train2, y_train, X_test2, y_test)
    print(start_month, end_month, model_num)
    model_num += 1
    start_month += 1
    end_month += 1
    print('tf_idf_nb')
    print(scores_tf_idf_nb)
    print('tf_idf_rf')
    print(scores_tf_idf_rf)

1 4 0
tf_idf_nb
[[0.52894464 0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.95906801 0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.27436498 0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.4266704  0.         0.         0.         0.         0.
  0.         0.         0.        ]]
tf_idf_rf
[[0.93336402 0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.68576826 0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.93156544 0.         0.         0.         0.         0.
  0.         0.         0.        ]
 [0.78998912 0.         0.         0.         0.         0.
  0.         0.         0.        ]]
2 5 1
tf_idf_nb
[[0.52894464 0.52224771 0.         0.         0.         0.
  0.         0.         0.        ]
 [0.95906801 0.94959042 0.         0.         0.         0.
  0.         0.         0.      

## TF IDF NAIVE BAYES

In [18]:
accuracy_2017_tf_idf_nb = np.mean(scores_tf_idf_nb[0])
recall_2017_tf_idf_nb = np.mean(scores_tf_idf_nb[1])
precision_2017_tf_idf_nb = np.mean(scores_tf_idf_nb[2])
f1_score_2017_tf_idf_nb = np.mean(scores_tf_idf_nb[3])
print(accuracy_2017_tf_idf_nb)
print(recall_2017_tf_idf_nb)
print(precision_2017_tf_idf_nb)
print(f1_score_2017_tf_idf_nb)

0.5474739954473133
0.9556813235672457
0.31188323987706895
0.46898189814778557


## TF IDF RANDOM FOREST

In [20]:
accuracy_2017_tf_idf_rf = np.mean(scores_tf_idf_rf[0])
recall_2017_tf_idf_rf = np.mean(scores_tf_idf_rf[1])
precision_2017_tf_idf_rf = np.mean(scores_tf_idf_rf[2])
f1_score_2017_tf_idf_rf = np.mean(scores_tf_idf_rf[3])
print(accuracy_2017_tf_idf_rf)
print(recall_2017_tf_idf_rf)
print(precision_2017_tf_idf_rf)
print(f1_score_2017_tf_idf_rf)

0.9222954413043342
0.6726863069580991
0.938235214202308
0.7830354690880177


In [ ]:
# Do I need k means? Or just top twenty words for popular and top twenty words for not popular

## Kmeans

In [21]:
from sklearn.cluster import KMeans
from collections import Counter
from scipy.spatial.distance import pdist, squareform
from scipy.cluster.hierarchy import linkage, dendrogram

In [23]:
features =  tf_idf_vectorizer_train.get_feature_names()

start_month = 1
end_month = 4
df_X_train = df_sf_2017[(df_sf_2017['month'] >= start_month) & (df_sf_2017['month'] < end_month)]['space_new']
y_train = df_sf_2017[(df_sf_2017['month'] >= start_month) & (df_sf_2017['month'] < end_month)]['popular']

df_X_test = df_sf_2017[df_sf_2017['month'] == end_month]['space_new']
y_test = df_sf_2017[df_sf_2017['month'] == end_month]['popular']

In [27]:
features[150:200]

['1300',
 '1300sqft',
 '1304',
 '1315',
 '132',
 '135',
 '1350',
 '135cm',
 '136',
 '137',
 '1370123',
 '13724120',
 '139',
 '1394',
 '1396',
 '13ft',
 '13in',
 '13lb',
 '13mn',
 '13th',
 '13x12',
 '14',
 '140',
 '1400',
 '1400ft²',
 '1400sf',
 '1408',
 '140sq',
 '140sqm',
 '14154147',
 '143',
 '144',
 '145',
 '1450',
 '14611926',
 '1474',
 '149',
 '14ft',
 '14l',
 '14min',
 '14r',
 '14sqm',
 '14th',
 '14x',
 '14x12',
 '15',
 '150',
 '1500',
 '1500ft',
 '1500sf']

In [26]:
%%time
kmeans = KMeans(n_clusters=2, n_jobs=-1)
kmeans.fit(X_train2)

CPU times: user 1min 17s, sys: 3.51 s, total: 1min 20s
Wall time: 1min 36s


In [224]:
%%time
# TF vectorizer
top_centroids = kmeans.cluster_centers_.argsort()[:,-1:-60:-1]
print("top features (words) for each cluster:")
for num, centroid in enumerate(top_centroids):
    print("%d: %s" % (num, ", ".join(features[i] for i in centroid)))


top features (words) for each cluster:
0: san, apartment, block, francisco, street, place, restaurant, neighborhood, park, kitchen, room, city, bedroom, walk, located, access, home, close, great, bed, ha, away, private, view, location, mission, space, sf, minute, 2, walking, bathroom, downtown, area, parking, house, bar, floor, distance, bus, bart, quiet, gate, easy, guest, available, district, public, golden, building, stay, 1, square, shop, just, unit, heart, youll, beautiful
1: room, bedroom, bed, kitchen, ha, bathroom, living, private, home, san, apartment, large, 2, access, francisco, house, floor, guest, street, queen, block, space, park, area, neighborhood, located, view, 1, city, restaurant, great, tv, parking, dining, quiet, walk, spacious, available, bath, shared, away, flat, beautiful, comfortable, sf, mission, garden, place, stay, size, 3, downtown, open, deck, easy, just, unit, close, new
CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 2.05 ms


In [31]:
%%time
# TF-IDF vectorizer
top_centroids = kmeans.cluster_centers_.argsort()[:,-1:-50:-1]
print("top features (words) for each cluster:")
for num, centroid in enumerate(top_centroids):
    print("%d: %s" % (num, ", ".join(features[i] for i in centroid)))


top features (words) for each cluster:
0: space, home, house, city, san, place, great, view, francisco, apartment, room, location, quiet, light, sf, floor, comfortable, ha, neighborhood, lot, close, restaurant, clean, unit, street, guest, beautiful, time, studio, park, modern, building, stay, feel, wa, like, kitchen, perfect, people, cozy, art, need, block, bed, unique, new, located, available, just
1: room, bedroom, bed, ha, kitchen, bathroom, living, private, apartment, large, floor, space, san, queen, home, 2, francisco, house, street, area, guest, tv, view, block, access, size, great, located, 1, city, comfortable, neighborhood, dining, park, spacious, bath, parking, flat, new, unit, quiet, away, closet, garden, window, walk, beautiful, coffee, available
CPU times: user 4 ms, sys: 0 ns, total: 4 ms
Wall time: 2.92 ms


In [28]:
def catPred(kmObj, gTrue, pred): 
    '''
        kmObjn: a kmeans object
        gTrue: true categories (ground truth)
    '''
    fig, ax = plt.subplots(nrows=1, ncols=2, sharey=True, figsize=(50,12))

    numClusters = kmObj.get_params()['n_clusters']
    
    for i in range(numClusters):
        print(i)
        mask = (pred == i)
        val = np.unique(gTrue[mask], return_counts=True)
        lbl = val[0]
        ht  = val[1] / val[1].sum()
        ax[i].bar(range(numClusters), height=ht, tick_label = lbl)
        ax[i].set_xticklabels(lbl, fontsize=20)
        ax[i].set_title(i, fontsize=20)

        ax[0].set_yticklabels(["0", "0.2", "0.4", "0.6", "0.8"], fontsize=20)
        ax[0].set_ylabel("Proportion", fontsize = 32);
        plt.suptitle("Proportion of Categories in each Cluster", fontsize = 32);

In [29]:
catPred(kmeans, y_test, preds)

NameError: name 'preds' is not defined

In [281]:
kmeans.get_params()['n_clusters']

2